In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import util
import os
import json

In [2]:
dir_grpc = 'data/grpc/'
dir_kafka = 'data/kafka/'

In [3]:
exps = dict()

In [4]:
with open('data/sizes_grouped_1000.json') as fp:
    size_map_1000 = json.load(fp)

In [5]:
# Load gRPC results
for filename in sorted(os.listdir(dir_grpc)):
    with open(dir_grpc + filename) as f:
        data = json.loads(f.read())
    kts = data['result']['size'] / np.array([
        float(ts['t1client'] - ts['t0server']) / 1_000_000
        for ts in data['result']['times']
    ])
    ds_name = util.get_ds_name(data)
    if ds_name == 'LOD':
        continue

    exps[(
        util.get_network_cond(data),
        'grpc_' + util.get_exp_name(data),
        ds_name,
    )] = kts

In [6]:
# Load Kafka results
for filename in sorted(os.listdir(dir_kafka)):
    with open(dir_kafka + filename) as fp:
        data = json.load(fp)

    ds_name = util.get_ds_name(data)
    if ds_name == 'LOD':
        continue
    size = size_map_1000[ds_name]

    kts = size / np.array([
        float(ts['t1cons'] - ts['t0prod']) / 1_000_000
        for ts in data['result']['times']
    ])

    exps[(
        util.get_network_cond(data),
        'kafka_' + util.get_exp_name(data),
        ds_name,
    )] = kts

In [7]:
dt_exps = pd.DataFrame.from_dict(exps)

In [8]:
means = dt_exps.iloc[5:].apply(np.mean, axis=0)
means_dt = means.reset_index().set_axis(['network', 'algo', 'dataset', 'mean_kts'], axis=1, inplace=False)
means_unst = means.unstack()
means_unst

aemet-1     aemet-2  flickr_10m  \
100Mbit   grpc_jelly_full            442.972210  297.123082  250.639592   
          grpc_jelly_full_gzip       361.057682  341.221992  220.637603   
          grpc_jelly_noprefix        447.502428  277.822399  239.250385   
          grpc_jelly_noprefix_gzip   427.051905  387.021363  244.544854   
          kafka_jelly_full           109.636418   49.186511   41.622764   
          kafka_jelly_full_gzip      396.367299  381.397191  218.877520   
          kafka_jelly_noprefix       109.280236   46.570035   41.421270   
          kafka_jelly_noprefix_gzip  475.845098  462.094515  220.542768   
          kafka_jena-proto            20.389427   15.717823   20.020739   
          kafka_jena-proto_gzip      319.063406  286.350447  176.363966   
          kafka_n3                    20.869068   17.347576   20.434417   
          kafka_n3_gzip              184.602624  140.646671  111.190435   
50Mbit    grpc_jelly_full            219.829314  140.839927  117.128198   
          grpc_jelly_full_gzip       344.388441  327.886599  215.868801   
          grpc_jelly_noprefix        220.035542  140.956309  114.535647   
          grpc_jelly_noprefix_gzip   427.809299  404.713549  243.672423   
          kafka_jelly_full            70.159039   31.160057   26.357675   
          kafka_jelly_full_gzip      391.505746  376.304753  139.563944   
          kafka_jelly_noprefix        68.170573   29.521428   26.196351   
          kafka_jelly_noprefix_gzip  419.626235  450.360308  140.711070   
          kafka_jena-proto            12.861914    9.957658   12.555687   
          kafka_jena-proto_gzip      226.868728  286.424546  109.739594   
          kafka_n3                    13.241827   10.932986   12.883068   
          kafka_n3_gzip              187.104862  144.841061  124.026392   
unlimited grpc_jelly_full            530.114929  457.052088  397.539634   
          grpc_jelly_full_gzip       349.095162  328.757582  221.465563   
          grpc_jelly_noprefix        677.502591  582.756747  465.093943   
          grpc_jelly_noprefix_gzip   426.507918  393.353099  242.283219   
          kafka_jelly_full           631.158839  577.982364  500.867257   
          kafka_jelly_full_gzip      414.442612  383.440342  242.855859   
          kafka_jelly_noprefix       863.659247  794.167250  515.626822   
          kafka_jelly_noprefix_gzip  482.435398  492.571208  263.990161   
          kafka_jena-proto           367.451574  323.428994  249.800372   
          kafka_jena-proto_gzip      325.014054  292.009959  227.131933   
          kafka_n3                   187.314223  136.787089  110.658247   
          kafka_n3_gzip              190.486013  140.469134  112.382973   

                                       identica  migr_reschange         mix  \
100Mbit   grpc_jelly_full            185.780502      412.559501  261.881951   
          grpc_jelly_full_gzip       154.170269      413.178863  219.036507   
          grpc_jelly_noprefix        176.513330      369.070496  241.933052   
          grpc_jelly_noprefix_gzip   166.949935      481.624163  242.616476   
          kafka_jelly_full            35.845826       75.631485   47.036551   
          kafka_jelly_full_gzip      110.710437      451.972260  173.791101   
          kafka_jelly_noprefix        34.700007       63.084007   41.372428   
          kafka_jelly_noprefix_gzip  110.971367      581.756885  174.488852   
          kafka_jena-proto            20.222977       15.232280   19.952976   
          kafka_jena-proto_gzip       98.726551      312.193105  167.253971   
          kafka_n3                    22.588032       16.667893   20.706489   
          kafka_n3_gzip               66.544144      141.182808   91.440153   
50Mbit    grpc_jelly_full            100.585022      195.583537  137.317675   
          grpc_jelly_full_gzip       149.200027      412.589405  222.468945   
          grpc_jelly_noprefix         94.299021      184.732865  123.059883   


In [9]:
def summarize_by_network(network: str) -> (pd.DataFrame, pd.DataFrame):
    s_full = means_dt[means_dt['network'] == network][['algo', 'dataset', 'mean_kts']]\
        .pivot(index='algo', columns='dataset', values='mean_kts')
    return s_full, s_full.apply(np.mean, axis=1).apply(lambda x: np.round(x, 2))


In [10]:
dt_unl_full, dt_unl = summarize_by_network('unlimited')
dt_unl

algo
grpc_jelly_full              398.55
grpc_jelly_full_gzip         270.11
grpc_jelly_noprefix          482.51
grpc_jelly_noprefix_gzip     314.91
kafka_jelly_full             480.73
kafka_jelly_full_gzip        322.18
kafka_jelly_noprefix         597.55
kafka_jelly_noprefix_gzip    391.37
kafka_jena-proto             282.33
kafka_jena-proto_gzip        257.09
kafka_n3                     125.15
kafka_n3_gzip                126.49
dtype: float64

In [11]:
dt_unl_full

dataset,aemet-1,aemet-2,flickr_10m,identica,migr_reschange,mix,nevada_10m,petrol,tour_cap_nuts3,wikipedia
algo,,,,,,,,,,
grpc_jelly_full,530.114929,457.052088,397.539634,273.997811,519.955252,358.693213,345.658661,195.630763,496.535674,410.316829
grpc_jelly_full_gzip,349.095162,328.757582,221.465563,165.399578,384.217639,223.307554,233.488978,190.205922,372.952518,232.211176
grpc_jelly_noprefix,677.502591,582.756747,465.093943,318.558885,681.203331,372.709885,412.080559,194.489311,655.220996,465.503407
grpc_jelly_noprefix_gzip,426.507918,393.353099,242.283219,182.943125,488.420079,249.451927,255.270398,182.787278,472.643254,255.392523
kafka_jelly_full,631.158839,577.982364,500.867257,296.062282,608.728507,403.756601,466.359054,179.083663,620.927163,522.385967
kafka_jelly_full_gzip,414.442612,383.440342,242.855859,193.442944,460.904805,251.941742,328.387820,176.442393,487.526922,282.385537
kafka_jelly_noprefix,863.659247,794.167250,515.626822,331.840058,888.804319,396.755030,538.588418,185.595405,846.486219,614.019424
kafka_jelly_noprefix_gzip,482.435398,492.571208,263.990161,212.034093,658.469946,288.846015,418.402158,176.112157,628.852429,291.964704
kafka_jena-proto,367.451574,323.428994,249.800372,201.297991,327.357498,248.387007,245.407712,162.201286,334.594689,363.336366


In [12]:
dt_100_full, dt_100 = summarize_by_network('100Mbit')
dt_100

algo
grpc_jelly_full              292.91
grpc_jelly_full_gzip         274.45
grpc_jelly_noprefix          269.76
grpc_jelly_noprefix_gzip     311.45
kafka_jelly_full              56.67
kafka_jelly_full_gzip        292.69
kafka_jelly_noprefix          52.75
kafka_jelly_noprefix_gzip    342.58
kafka_jena-proto              18.74
kafka_jena-proto_gzip        227.28
kafka_n3                      20.16
kafka_n3_gzip                120.35
dtype: float64

In [13]:
dt_100_full

dataset,aemet-1,aemet-2,flickr_10m,identica,migr_reschange,mix,nevada_10m,petrol,tour_cap_nuts3,wikipedia
algo,,,,,,,,,,
grpc_jelly_full,442.972210,297.123082,250.639592,185.780502,412.559501,261.881951,225.145133,192.913282,408.847014,251.246049
grpc_jelly_full_gzip,361.057682,341.221992,220.637603,154.170269,413.178863,219.036507,238.755561,186.930270,373.242915,236.311282
grpc_jelly_noprefix,447.502428,277.822399,239.250385,176.513330,369.070496,241.933052,174.048553,193.097877,361.523067,216.866175
grpc_jelly_noprefix_gzip,427.051905,387.021363,244.544854,166.949935,481.624163,242.616476,251.491731,193.138018,464.361416,255.666853
kafka_jelly_full,109.636418,49.186511,41.622764,35.845826,75.631485,47.036551,49.062746,52.549832,64.454984,41.697711
kafka_jelly_full_gzip,396.367299,381.397191,218.877520,110.710437,451.972260,173.791101,337.219660,174.169054,468.402007,214.019072
kafka_jelly_noprefix,109.280236,46.570035,41.421270,34.700007,63.084007,41.372428,46.609855,41.790658,62.061167,40.573059
kafka_jelly_noprefix_gzip,475.845098,462.094515,220.542768,110.971367,581.756885,174.488852,429.415636,178.611447,578.055439,214.045400
kafka_jena-proto,20.389427,15.717823,20.020739,20.222977,15.232280,19.952976,14.062467,20.011481,15.206921,26.548866


In [14]:
dt_50_full, dt_50 = summarize_by_network('50Mbit')
dt_50

algo
grpc_jelly_full              150.12
grpc_jelly_full_gzip         268.22
grpc_jelly_noprefix          138.43
grpc_jelly_noprefix_gzip     311.79
kafka_jelly_full              35.91
kafka_jelly_full_gzip        272.03
kafka_jelly_noprefix          33.34
kafka_jelly_noprefix_gzip    304.86
kafka_jena-proto              11.85
kafka_jena-proto_gzip        186.61
kafka_n3                      12.73
kafka_n3_gzip                123.27
dtype: float64

In [15]:
dt_50_full

dataset,aemet-1,aemet-2,flickr_10m,identica,migr_reschange,mix,nevada_10m,petrol,tour_cap_nuts3,wikipedia
algo,,,,,,,,,,
grpc_jelly_full,219.829314,140.839927,117.128198,100.585022,195.583537,137.317675,109.405873,157.032122,200.226651,123.278602
grpc_jelly_full_gzip,344.388441,327.886599,215.868801,149.200027,412.589405,222.468945,233.200550,184.991532,361.149245,230.459603
grpc_jelly_noprefix,220.035542,140.956309,114.535647,94.299021,184.732865,123.059883,87.095834,127.278783,182.223287,110.129109
grpc_jelly_noprefix_gzip,427.809299,404.713549,243.672423,160.021925,480.513400,242.460932,252.927912,190.831087,458.160953,256.747240
kafka_jelly_full,70.159039,31.160057,26.357675,23.066848,46.929006,30.019641,31.141596,33.284102,40.518830,26.465217
kafka_jelly_full_gzip,391.505746,376.304753,139.563944,75.993725,468.639080,118.843137,339.962944,189.763601,476.456304,143.228800
kafka_jelly_noprefix,68.170573,29.521428,26.196351,22.312736,39.911082,26.441093,29.369092,26.507672,39.307774,25.631454
kafka_jelly_noprefix_gzip,419.626235,450.360308,140.711070,76.082461,589.006886,118.639333,339.256994,179.298828,592.297375,143.298367
kafka_jena-proto,12.861914,9.957658,12.555687,12.930958,9.666884,12.621798,8.939839,12.551787,9.652311,16.721770


In [16]:
dt_networks = pd.DataFrame(data=(dt_unl, dt_100, dt_50)).T
dt_networks.columns = ['unlimited', '100Mbit', '50Mbit']
dt_networks

,unlimited,100Mbit,50Mbit
algo,,,
grpc_jelly_full,398.55,292.91,150.12
grpc_jelly_full_gzip,270.11,274.45,268.22
grpc_jelly_noprefix,482.51,269.76,138.43
grpc_jelly_noprefix_gzip,314.91,311.45,311.79
kafka_jelly_full,480.73,56.67,35.91
kafka_jelly_full_gzip,322.18,292.69,272.03
kafka_jelly_noprefix,597.55,52.75,33.34
kafka_jelly_noprefix_gzip,391.37,342.58,304.86
kafka_jena-proto,282.33,18.74,11.85


In [20]:
dt_unl_full.to_latex(
    'extra/latex/full_unl.tex',
    float_format='%.2f',
    caption='End-to-End Streaming Throughput -- unlimited network (kT/s)',
)
dt_100_full.to_latex(
    'extra/latex/full_100.tex',
    float_format='%.2f',
    caption='End-to-End Streaming Throughput -- 100\,Mbit/s network (kT/s)',
)
dt_50_full.to_latex(
    'extra/latex/full_50.tex',
    float_format='%.2f',
    caption='End-to-End Streaming Throughput -- 50\,Mbit/s network (kT/s)',
)

/tmp/ipykernel_230919/1125886907.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  dt_unl_full.to_latex(
/tmp/ipykernel_230919/1125886907.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  dt_100_full.to_latex(
/tmp/ipykernel_230919/1125886907.py:11: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead